In [6]:
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(100,100))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(100,100))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(100,100))
    images=[row,scale_out,scale_in,rot]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,100,100,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons,conv_layers_continuous):
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            for i in range (conv_layers_continuous[0]):
                hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
            conv_layers_continuous.pop(0)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,128],[50,50],[2,3,3])

[1493, 2780, 1583]
3747
14988
Epoch 1/100
  4/100 [>.............................] - ETA: 14:02 - loss: 1.1076 - accuracy: 0.4267

KeyboardInterrupt: 

In [2]:
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

import random as ra

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(200,200))
    images=[row,scale_out,scale_in,rot]
    images_selected=[row,images[ra.randint(1,3)]]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim200x200'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        keeped=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        keeped_Y=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        if(i==0):
            keeped=keeped[0:int(len(keeped)/2)]
            keeped_Y=keeped_Y[0:int(len(keeped_Y)/2)]                 
        TVT[i]+=keeped
        Y_TVT[i]+=keeped_Y
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,200,200,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons,conv_layers_continuous):
    
    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            for i in range (conv_layers_continuous[0]):
                hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='same',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
            conv_layers_continuous.pop(0)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            hidden=layers.BatchNormalization()(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max',restore_best_weights=True)
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len
              ,steps_per_epoch=20,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,128,256],[40,30],[2,2,2,2])

[1493, 2780, 1583]
3747
7494
Epoch 1/100
20/20 [==============================] - 540s 26s/step - loss: 1.4307 - accuracy: 0.3440 - val_loss: 6.5534 - val_accuracy: 0.2700
Epoch 2/100
20/20 [==============================] - 556s 28s/step - loss: 1.2116 - accuracy: 0.4127 - val_loss: 1.8568 - val_accuracy: 0.4749
Epoch 3/100
20/20 [==============================] - 658s 33s/step - loss: 1.1672 - accuracy: 0.4342 - val_loss: 1.9277 - val_accuracy: 0.6137
Epoch 4/100
20/20 [==============================] - 592s 30s/step - loss: 1.1203 - accuracy: 0.4610 - val_loss: 1.4294 - val_accuracy: 0.5208
Epoch 5/100
20/20 [==============================] - 593s 30s/step - loss: 1.0950 - accuracy: 0.4699 - val_loss: 0.8279 - val_accuracy: 0.6766
Epoch 6/100
20/20 [==============================] - 590s 30s/step - loss: 1.0603 - accuracy: 0.4930 - val_loss: 0.8796 - val_accuracy: 0.6617
Epoch 7/100
20/20 [==============================] - 594s 30s/step - loss: 1.0384 - accuracy: 0.5017 - val_loss: 

In [3]:
#reduce L punishment 10,-6
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

import random as ra

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(200,200))
    images=[row,scale_out,scale_in,rot]
    images_selected=[row,images[ra.randint(1,3)]]
    for image in images_selected:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim200x200'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        
Augmented=[]
Y_Augmented=[]

for i in range(len(Train)):
    augmentation(Train[i],Y_Train[i],Augmented,Y_Augmented)

print(len(Train))

TVT[0]=Augmented
Y_TVT[0]=Y_Augmented

for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,200,200,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons,conv_layers_continuous):
    
    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            for i in range (conv_layers_continuous[0]):
                hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='same',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
            conv_layers_continuous.pop(0)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(pow(10,-6)))(hidden)
            hidden=layers.BatchNormalization()(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(pow(10,-6)))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len
              ,steps_per_epoch=20,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,128,256],[40,30],[2,2,2,2])

[1493, 2780, 1583]
3747
7494
Epoch 1/100
20/20 [==============================] - 647s 32s/step - loss: 1.3883 - accuracy: 0.3707 - val_loss: 1.6334 - val_accuracy: 0.1793
Epoch 2/100
20/20 [==============================] - 607s 30s/step - loss: 1.2852 - accuracy: 0.3777 - val_loss: 2.0960 - val_accuracy: 0.2668
Epoch 3/100
20/20 [==============================] - 625s 31s/step - loss: 1.1656 - accuracy: 0.4299 - val_loss: 1.3119 - val_accuracy: 0.2892
Epoch 4/100
20/20 [==============================] - 620s 31s/step - loss: 1.1555 - accuracy: 0.4350 - val_loss: 1.3879 - val_accuracy: 0.2711
Epoch 5/100
20/20 [==============================] - 620s 31s/step - loss: 1.1321 - accuracy: 0.4522 - val_loss: 1.2419 - val_accuracy: 0.3116
Epoch 6/100
20/20 [==============================] - 600s 30s/step - loss: 1.1060 - accuracy: 0.4673 - val_loss: 2.6014 - val_accuracy: 0.3714
Epoch 7/100
20/20 [==============================] - 623s 31s/step - loss: 1.0581 - accuracy: 0.4863 - val_loss: 

In [5]:
#reduce L punishment 10,-6
def process(X,file):
    img=cv2.imread(file,cv2.IMREAD_GRAYSCALE)
    X.append(img)
    #write your code

def normalize(dataset):
    dataset=asarray(dataset).astype('float32')
    dataset=dataset/255.0
    return dataset

import random as ra

def augmentation(row,label,Augmented,Y_Augmeneted):
    scale_out = skimage.transform.rescale(row, scale=2.0, mode='constant')
    scale_out=skimage.transform.resize(scale_out,(200,200))
    scale_in= skimage.transform.rescale(row, scale=0.5, mode='constant')
    scale_in=skimage.transform.resize(scale_in,(200,200))
    rot = skimage.transform.rotate(row, angle=20, mode='reflect')
    rot=skimage.transform.resize(rot,(200,200))
    images=[row,scale_out,scale_in,rot]
    #images_selected=[row,images[ra.randint(1,3)]]
    for image in images:
        Augmented.append(image)
        Y_Augmented.append(label)



import skimage
from skimage import transform
from skimage.transform import resize
from skimage.transform import rescale
import cv2
import glob
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim200x200'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        keeped=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        keeped_Y=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]             
        TVT[i]+=keeped
        Y_TVT[i]+=keeped_Y
        


print(len(Train))


for i in range(3):
    TVT[i]=normalize(TVT[i])
    TVT[i]=TVT[i].reshape(-1,200,200,1)
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)

print(len(Augmented))

# epoch=32 steps_per_epoch=32
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train(epochs_num,batch_len,conv_neurons,art_neurons,conv_layers_continuous):
    
    def mlp_model():
        Inp=layers.Input(shape=(200,200,1))
        hidden=Inp
        for conv_neuron in conv_neurons:
            for i in range (conv_layers_continuous[0]):
                hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='same',activation='relu')(hidden)
                hidden=layers.BatchNormalization()(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
            conv_layers_continuous.pop(0)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(pow(10,-7)))(hidden)
            hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(pow(10,-7)))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20,mode='max',restore_best_weights=True)
    model=mlp_model()
    model.fit(TVT[0],Y_TVT[0],validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,batch_size=batch_len
              ,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,50,[32,64,128,256],[40,30],[1,1,2,2])

[1493, 2780, 1583]
3747
3740
Epoch 1/100
75/75 [==============================] - 464s 6s/step - loss: 1.5106 - accuracy: 0.5490 - val_loss: 1.4036 - val_accuracy: 0.2615
Epoch 2/100
75/75 [==============================] - 452s 6s/step - loss: 0.9427 - accuracy: 0.5733 - val_loss: 1.1041 - val_accuracy: 0.3287
Epoch 3/100
75/75 [==============================] - 458s 6s/step - loss: 0.8602 - accuracy: 0.6293 - val_loss: 1.0754 - val_accuracy: 0.4568
Epoch 4/100
75/75 [==============================] - 448s 6s/step - loss: 0.8632 - accuracy: 0.6069 - val_loss: 1.1206 - val_accuracy: 0.4749
Epoch 5/100
75/75 [==============================] - 451s 6s/step - loss: 1.0026 - accuracy: 0.4654 - val_loss: 1.0356 - val_accuracy: 0.4749
Epoch 6/100
75/75 [==============================] - 449s 6s/step - loss: 0.9474 - accuracy: 0.4876 - val_loss: 1.6152 - val_accuracy: 0.4749
Epoch 7/100
75/75 [==============================] - 454s 6s/step - loss: 0.7771 - accuracy: 0.6205 - val_loss: 0.8230 

75/75 [==============================] - 450s 6s/step - loss: 0.3424 - accuracy: 0.8393 - val_loss: 0.6101 - val_accuracy: 0.7150
Epoch 59/100
75/75 [==============================] - 449s 6s/step - loss: 0.3918 - accuracy: 0.8209 - val_loss: 0.6652 - val_accuracy: 0.7439
Epoch 60/100
75/75 [==============================] - 451s 6s/step - loss: 0.3661 - accuracy: 0.8287 - val_loss: 0.6382 - val_accuracy: 0.7684
Epoch 61/100
75/75 [==============================] - 452s 6s/step - loss: 0.3438 - accuracy: 0.8340 - val_loss: 0.6618 - val_accuracy: 0.7748
Epoch 62/100
75/75 [==============================] - 452s 6s/step - loss: 0.3570 - accuracy: 0.8340 - val_loss: 0.7121 - val_accuracy: 0.7791
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 200, 200, 1)]     0         
                                                                 
 conv2d_24 (Conv2

[2, 2, 2]
